In [14]:
from pymongo.mongo_client import MongoClient

client = MongoClient(host="localhost", port=27017)
job_col = client.Cacher.Job
client_col = client.Cacher.Client

In [15]:
client_info = [
    {
        "username": "user1",
        "jobs": ["job1", 'job2']
    },
    {
        "username": "user2",
        "jobs": ["job3", 'job4']
    }
]

In [16]:
client_col.insert_many(client_info)

In [52]:
jobInfo = [
    {
        "meta": {
            "username": "zhuangwei",
            "jobId": "j1"
        },
        "QoS": {
            "useCache": True
        },
        "policy": {
            "chunkSize": 1024,
            "chunks": [{"key": 1, "location": "redis", "hasBackup": False}, {"key": 2, "location": "disk", "hasBackup": False}]
        }
    },
    {
        "meta": {
            "username": "zhuangwei",
            "jobId": "j2"
        },
        "QoS": {
            "useCache": True
        },
        "policy": {
            "chunkSize": 1024,
            "chunks": [{"key": 4, "location": "redis", "hasBackup": False}, {"key": 5, "location": "disk", "hasBackup": False}]
        }
    },
    {
        "meta": {
            "username": "zhuangwei",
            "jobId": "j3"
        },
        "QoS": {
            "useCache": True
        },
        "policy": {
            "chunkSize": 1024,
            "chunks": [{"key": 3, "location": "redis", "hasBackup": True}, {"key": 6, "location": "disk", "hasBackup": False}]
        }
    }
]

In [53]:
job_col.insert_many(jobInfo)

In [88]:
pipeline = [
    {"$unwind": "$policy.chunks"},
    {"$project": {
        "_id": 0, 
        "key": "$policy.chunks.key", 
        "hasBackup": "$policy.chunks.hasBackup",
        "chunk": {"$regexFind": {"input": "$policy.chunks.location", "regex": "^redis", "options": "m"}}}
    },
    {"$match": {"chunk": {"$ne": None}}},
    {"$sort": {"key": 1}},
    {"$limit": 3},
    {"$match": {"hasBackup": {"$eq": False}}},
    {"$project": {"key": 1}},
    {"$group": {"_id": "$_id", "keys": {"$push": "$key"}}},
]

backups = job_col.aggregate(pipeline).next()['keys']
backups

[1, 4]

In [116]:
rlt = job_col.update_many(
    {"policy.chunks": {"$elemMatch": {"key": {"$in": backups}}}},
    {"$set": {"policy.chunks.$.hasBackup": True}}
)

print(rlt)